<a href="https://colab.research.google.com/github/Aishwarya-M-Shetty/TensorFlow/blob/main/Mnist_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [ ]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 7s - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980 - 7s/epoch - 4ms/step
Epoch 2/10
1719/1719 - 5s - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596 - 5s/epoch - 3ms/step
Epoch 3/10
1719/1719 - 6s - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834 - 6s/epoch - 4ms/step
Epoch 4/10
1719/1719 - 5s - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940 - 5s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 5s - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006 - 5s/epoch - 3ms/step
Epoch 6/10
1719/1719 - 5s - loss: 0.3852 - accuracy: 0.8938 - val_loss: 0.3525 - val_accuracy: 0.9052 - 5s/epoch - 3ms/step
Epoch 7/10
1719/1719 - 5s - loss: 0.3644 - accuracy: 0.8979 - val_loss: 0.3348 - val_accuracy: 0.9102 - 5s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 5s - loss: 0.3485 - accuracy: 0.9021 - val_loss: 0.3209 - val_accuracy: 0.9138 - 5s/epoch - 3ms/step
Epoch 9/

In [ ]:
model.save("pretrained_mnist_model.h5")

In [ ]:
#Classify handwritten digits into odd and even
pretrained_mnist_model = tf.keras.models.load_model("pretrained_mnist_model.h5")

In [ ]:
pretrained_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [ ]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: True
dense: True
leaky_re_lu: True
dense_1: True
leaky_re_lu_1: True
dense_2: True


In [ ]:
for layer in pretrained_mnist_model.layers[:-1]: # leaves the last layer unfreezed => trainable
  layer.trainable = False # freezing the layers from getting trained
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False


In [ ]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False
dense_2: True


In [ ]:
try:
  del new_model
except:
  pass

In [ ]:
lower_pretrained_layers = pretrained_mnist_model.layers[:-1]

# LAYERS = [
#           lower_pretrained_layers,
#           tf.keras.layers.Dense(2, activation="softmax")
# ]

new_model = tf.keras.models.Sequential(lower_pretrained_layers)

new_model.add(
    tf.keras.layers.Dense(2, activation="softmax")
)

In [ ]:
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265,802
Trainable params: 202
Non-traina

In [ ]:
np.where(y_train%2 == 0, 1, 0)

array([0, 0, 1, ..., 0, 1, 1])

In [ ]:
y_train[0], y_train[-1]

(7, 8)

In [ ]:
def update_even_odd_labels(labels):
  for idx, label in enumerate(labels):
    labels[idx] = np.where(label % 2 == 0, 1, 0)
  return labels

In [ ]:
y_train_bin, y_test_bin, y_valid_bin = update_even_odd_labels([y_train, y_test, y_valid])

In [ ]:
np.unique(y_train_bin)

array([0, 1])

In [ ]:
new_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
history = new_model.fit(X_train, y_train_bin, epochs=10,
                    validation_data=(X_valid, y_valid_bin), verbose=2)

Epoch 1/10
1719/1719 - 7s - loss: 0.4272 - accuracy: 0.8015 - val_loss: 0.3464 - val_accuracy: 0.8510 - 7s/epoch - 4ms/step
Epoch 2/10
1719/1719 - 5s - loss: 0.3465 - accuracy: 0.8489 - val_loss: 0.3183 - val_accuracy: 0.8650 - 5s/epoch - 3ms/step
Epoch 3/10
1719/1719 - 5s - loss: 0.3269 - accuracy: 0.8591 - val_loss: 0.3027 - val_accuracy: 0.8706 - 5s/epoch - 3ms/step
Epoch 4/10
1719/1719 - 5s - loss: 0.3151 - accuracy: 0.8651 - val_loss: 0.2927 - val_accuracy: 0.8776 - 5s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 5s - loss: 0.3065 - accuracy: 0.8697 - val_loss: 0.2860 - val_accuracy: 0.8836 - 5s/epoch - 3ms/step
Epoch 6/10
1719/1719 - 5s - loss: 0.3000 - accuracy: 0.8726 - val_loss: 0.2791 - val_accuracy: 0.8846 - 5s/epoch - 3ms/step
Epoch 7/10
1719/1719 - 5s - loss: 0.2946 - accuracy: 0.8758 - val_loss: 0.2750 - val_accuracy: 0.8878 - 5s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 4s - loss: 0.2902 - accuracy: 0.8782 - val_loss: 0.2701 - val_accuracy: 0.8888 - 4s/epoch - 3ms/step
Epoch 9/

In [ ]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 3ms/step - loss: 0.2797 - accuracy: 0.8847


[0.2797388732433319, 0.8847000002861023]

In [ ]:
X_new = X_test[:3]
y_test[:3], y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([0, 1, 0]))

In [ ]:
np.argmax(new_model.predict(X_new), axis=-1)

array([0, 1, 0])